<a href="https://colab.research.google.com/github/nxxk23/AI-Engineer/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install gradio langchain elasticsearch transformers sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.2/320.2 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 543.1/543.1 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.2/168.2 kB 6.0 MB/s eta 0:00:00


In [2]:
import gradio as gr
import json
import ast
import requests
import logging
from datetime import datetime
import re
from elasticsearch import Elasticsearch
from transformers import AutoTokenizer, AutoModel
import ast
import logging
import time
import re
import pandas as pd

In [3]:
user_e = "natthaphol.po"
passwords_e = "cnEM5CeFrG"

es = Elasticsearch(
    hosts=[{'host': 'elk.manageai.co.th', 'port': 443, 'scheme': 'https'}],
    basic_auth=(user_e, passwords_e)
)

url = "https://ai-api.manageai.co.th/llm-model-02/generate"
url2 = "https://ai-api.manageai.co.th/llm-model-03/generate"
headers = {'Content-Type': 'application/json'}
username = 'manageai2024'
passwords = 'ManageAI@2024'

tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
model = AutoModel.from_pretrained('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

model_param02 = {
    "max_new_tokens": 70,
    "repetition_penalty": 0.98,
    "stop": ["คำถาม","*","/end"],
}

model_param03 = {
    "max_new_tokens": 120,
    "repetition_penalty": 0.98,
    "stop": ["คำถาม",],
}

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

In [20]:
def generate_response(prompt, model_param, url):
    auth=(username, passwords)
    data = json.dumps({"inputs": prompt, "parameters": model_param})
    response = requests.post(url, data=data, headers=headers ,auth=auth)
    if response.status_code == 200:
        response_json = response.json()
        if "generated_text" in response_json:
            return response_json["generated_text"]
        else:
            print("The 'generated_text' field is missing.")
            return ""
    else:
        print(f"Request failed with status code {response.status_code}")
        print("Response content:", response.content)
        return ""

## ข้อมูล

In [85]:
# Predefined patient and policy data
patients = {
    "case 1": {
        "id_card": "0123456789012",
        "title": "นาย",
        "first_name": "สมชาย",
        "last_name": "ใจดี",
        "birth_date": "1973-01-01",
        "visit_date": "2024-08-01",
        "treatment_type": "เจ็บป่วย",
        "visit_type": "เข้ารับการรักษาครั้งแรก",
        "chief_complaint": "นัดมาตรวจซ้ำ...รับยา",
        "diagnosis": "I10 ความดันโลหิตสูงไม่ทราบสาเหตุ (ปฐมภูมิ) (Essential (primary) hypertension)",
        "billings": "[{'billing_initial': '6301.00', 'billing_discount': '630.00', 'simb_billing_code': '1.1.1(3)', 'billing_net_amount': '5671.00', 'local_billing_code': '50', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'billing_initial': '185', 'billing_discount': '0', 'simb_billing_code': '1.1.12', 'billing_net_amount': '185.00', 'local_billing_code': '240', 'local_billing_name': 'Nursing and Midwifery Charge   ค่าบริการพยาบาล'}, {'billing_initial': '185', 'billing_discount': '0', 'simb_billing_code': '1.1.14(2)', 'billing_net_amount': '185.00', 'local_billing_code': '280', 'local_billing_name': 'Other Medical Service Charge ค่าบริการทางการแพทย์'}, {'billing_initial': '800', 'billing_discount': '0', 'simb_billing_code': '1.2.1(1)', 'billing_net_amount': '800.00', 'local_billing_code': '310', 'local_billing_name': 'First Outpatient care before admission  ค่าแพทย์ตรวจรักษาครั้งแรก'}]",
        "order_items": "[{'initial': '863', 'item_id': '0102F02B0016O01', 'discount': '86.3', 'item_name': 'Blopress 16 mg', 'net_amount': '776.70', 'item_amount': '10', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': '50', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '1823', 'item_id': '0102E06I0240O05', 'discount': '182.3', 'item_name': 'Isoptin==SR== ๒๔๐ mg # (กส) ( VERAPAMIL)', 'net_amount': '1640.70', 'item_amount': '30', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': '50', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '750', 'item_id': '0102E04D0030L02', 'discount': '75', 'item_name': 'Dilatam 30 mg (กส)', 'net_amount': '675.00', 'item_amount': '60', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': '50', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '210', 'item_id': '0102N02A0081L02', 'discount': '21', 'item_name': 'Baby ASA  81  mg ( Enteric coat)', 'net_amount': '189.00', 'item_amount': '30', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': '50', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '2655', 'item_id': '0110E14P0002O01', 'discount': '265.4', 'item_name': 'Livalo 2 mg(กส)', 'net_amount': '2389.60', 'item_amount': '30', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': '50', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '185', 'item_id': 'NSE', 'discount': '0', 'item_name': 'ค่าบริการพยาบาล ผู้ป่วยนอก (Out Patient Service Charge)', 'net_amount': '185.00', 'item_amount': '1', 'simb_billing_code': '1.1.12', 'local_billing_code': '240', 'local_billing_name': 'Nursing and Midwifery Charge   ค่าบริการพยาบาล'}, {'initial': '185', 'item_id': 'NH', 'discount': '0', 'item_name': 'ค่าบริการโรงพยาบาล OPD (OPD Hospital Service Charge)', 'net_amount': '185.00', 'item_amount': '1', 'simb_billing_code': '1.1.14(2)', 'local_billing_code': '280', 'local_billing_name': 'Other Medical Service Charge ค่าบริการทางการแพทย์'}, {'initial': '800', 'item_id': 'DF1', 'discount': '0', 'item_name': 'DF แพทย์ตรวจรักษา (Doctor Fee)', 'net_amount': '800.00', 'item_amount': '1', 'simb_billing_code': '1.2.1(1)', 'local_billing_code': '310', 'local_billing_name': 'First Outpatient care before admission  ค่าแพทย์ตรวจรักษาครั้งแรก'}]",
        "total_net": "6841.0"
    },
    "case 2": {
        "id_card": "9876543210987",
        "title": "นางสาว",
        "first_name": "สวยใส",
        "last_name": "ดีใจ",
        "birth_date": "1990-05-15",
        "visit_date": "2024-09-01",
        "treatment_type": "เจ็บป่วย",
        "visit_type": "เข้ารับการรักษาครั้งแรก",
        "chief_complaint": "หูอื้อสองข้าง 1  วัน",
        "diagnosis": "H699 ความผิดของท่อยูสเตเชียน ไม่ระบุรายละเอียด (Eustachian tube disorder, unspecified)",
        "billings": "[{'billing_initial': '155', 'billing_discount': '0', 'simb_billing_code': '1.1.12', 'billing_net_amount': '155.00', 'local_billing_code': '121013', 'local_billing_name': 'Nursing and Midwifery Charge   ค่าบริการพยาบาล'}, {'billing_initial': '125', 'billing_discount': '0', 'simb_billing_code': '1.1.14(2)', 'billing_net_amount': '125.00', 'local_billing_code': '140100', 'local_billing_name': 'Other Medical Service Charge ค่าบริการทางการแพทย์'}, {'billing_initial': '100', 'billing_discount': '0', 'simb_billing_code': '1.1.7(1)', 'billing_net_amount': '100.00', 'local_billing_code': '710301', 'local_billing_name': 'General Medical Equipment  ค่าอุปกรณ์นอกห้องผ่าตัด'}, {'billing_initial': '700', 'billing_discount': '0', 'simb_billing_code': '1.2.1(1)', 'billing_net_amount': '700.00', 'local_billing_code': '11', 'local_billing_name': 'First Outpatient care before admission  ค่าแพทย์ตรวจรักษาครั้งแรก'}, {'billing_initial': '147', 'billing_discount': '13.8', 'simb_billing_code': '1.1.1(3)', 'billing_net_amount': '133.20', 'local_billing_code': 'PTFEX02', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'billing_initial': '140', 'billing_discount': '14', 'simb_billing_code': '1.1.1(3)', 'billing_net_amount': '126.00', 'local_billing_code': 'PTPSE02', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'billing_initial': '112', 'billing_discount': '11.2', 'simb_billing_code': '1.1.1(3)', 'billing_net_amount': '100.80', 'local_billing_code': 'PTPRE01', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}]",
        "order_items": "[{'initial': '155', 'item_id': '121013', 'discount': '0', 'item_name': 'ค่าบริการพยาบาลผู้ป่วยนอก', 'net_amount': '155.00', 'item_amount': '1', 'simb_billing_code': '1.1.12', 'local_billing_code': '121013', 'local_billing_name': 'Nursing and Midwifery Charge   ค่าบริการพยาบาล'}, {'initial': '125', 'item_id': '140100', 'discount': '0', 'item_name': 'ค่าบริการทางการแพทย์ผู้ป่วยนอก', 'net_amount': '125.00', 'item_amount': '1', 'simb_billing_code': '1.1.14(2)', 'local_billing_code': '140100', 'local_billing_name': 'Other Medical Service Charge ค่าบริการทางการแพทย์'}, {'initial': '100', 'item_id': '710301', 'discount': '0', 'item_name': 'การใช้เครื่องมือตรวจ EENT (SMR)', 'net_amount': '100.00', 'item_amount': '1', 'simb_billing_code': '1.1.7(1)', 'local_billing_code': '710301', 'local_billing_name': 'General Medical Equipment  ค่าอุปกรณ์นอกห้องผ่าตัด'}, {'initial': '700', 'item_id': '11', 'discount': '0', 'item_name': 'ค่าตรวจรักษากรณีผู้ป่วยนอก', 'net_amount': '700.00', 'item_amount': '1', 'simb_billing_code': '1.2.1(1)', 'local_billing_code': '11', 'local_billing_name': 'First Outpatient care before admission  ค่าแพทย์ตรวจรักษาครั้งแรก'}, {'initial': '147', 'item_id': 'PTFEX02', 'discount': '13.8', 'item_name': 'FEXOFENADINE 180 MG (FENAFEX) (A)', 'net_amount': '133.20', 'item_amount': '7', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': 'PTFEX02', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '140', 'item_id': 'PTPSE02', 'discount': '14', 'item_name': 'PSEUDOEPHEDRINE 60 MG.TABLET  (A)', 'net_amount': '126.00', 'item_amount': '20', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': 'PTPSE02', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '112', 'item_id': 'PTPRE01', 'discount': '11.2', 'item_name': 'PREDNISOLONE (FOIL) 5 MG. tablet(B)', 'net_amount': '100.80', 'item_amount': '28', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': 'PTPRE01', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}]",
        "total_net": "1440.0",
    },
    "case 3": {
        "id_card": "1111111111111",
        "title": "นางสาว",
        "first_name": "กุ๊กกุ๊ก",
        "last_name": "ไก๊ไก่",
        "birth_date": "1973-01-01",
        "visit_date": "2024-08-01",
        "treatment_type": "เจ็บป่วย",
        "visit_type": "เข้ารับการรักษาครั้งแรก",
        "chief_complaint": "ปวดหัว 3 วัน ไม่มีคลื่นไส้",
        "diagnosis": "R510 ปวดศีรษะ ไม่ระบุรายละเอียด (Headache, unspecified (TM))",
        "billings": "[{'billing_initial': '430.00', 'billing_discount': '43.00', 'simb_billing_code': '1.1.1(3)', 'billing_net_amount': '387.00', 'local_billing_code': '50', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'billing_initial': '175', 'billing_discount': '0', 'simb_billing_code': '1.1.12', 'billing_net_amount': '175.00', 'local_billing_code': '240', 'local_billing_name': 'Nursing and Midwifery Charge   ค่าบริการพยาบาล'}, {'billing_initial': '175.00', 'billing_discount': '0.00', 'simb_billing_code': '1.1.14(2)', 'billing_net_amount': '175.00', 'local_billing_code': '280', 'local_billing_name': 'Other Medical Service Charge ค่าบริการทางการแพทย์'}, {'billing_initial': '720', 'billing_discount': '0', 'simb_billing_code': '1.2.1(1)', 'billing_net_amount': '720.00', 'local_billing_code': '310', 'local_billing_name': 'First Outpatient care before admission  ค่าแพทย์ตรวจรักษาครั้งแรก'}]",
        "order_items": "[{'initial': '170', 'item_id': '0104B04T0500O06', 'discount': '17', 'item_name': 'TYLENOL 500 MG TAB*ก', 'net_amount': '153.00', 'item_amount': '20', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': '50', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '260', 'item_id': '0104C17N0250L01', 'discount': '26', 'item_name': 'NAPROXEN 250 MG TAB* [D3]', 'net_amount': '234.00', 'item_amount': '20', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': '50', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '175', 'item_id': 'NSOP', 'discount': '0', 'item_name': 'บริการพยาบาล OPD', 'net_amount': '175.00', 'item_amount': '1', 'simb_billing_code': '1.1.12', 'local_billing_code': '240', 'local_billing_name': 'Nursing and Midwifery Charge   ค่าบริการพยาบาล'}, {'initial': '175', 'item_id': 'S-NH0P', 'discount': '0', 'item_name': 'ค่าบริการโรงพยาบาล OPD', 'net_amount': '175.00', 'item_amount': '1', 'simb_billing_code': '1.1.14(2)', 'local_billing_code': '280', 'local_billing_name': 'Other Medical Service Charge ค่าบริการทางการแพทย์'}, {'initial': '0', 'item_id': 'ADS-112', 'discount': '0', 'item_name': 'Adm.score 11-12', 'net_amount': '0.00', 'item_amount': '1', 'simb_billing_code': '1.1.14(2)', 'local_billing_code': '280', 'local_billing_name': 'Other Medical Service Charge ค่าบริการทางการแพทย์'}, {'initial': '720', 'item_id': 'DFO-001', 'discount': '0', 'item_name': 'DF แพทย์ตรวจรักษา (Doctor Fee)', 'net_amount': '720.00', 'item_amount': '1', 'simb_billing_code': '1.2.1(1)', 'local_billing_code': '310', 'local_billing_name': 'First Outpatient care before admission  ค่าแพทย์ตรวจรักษาครั้งแรก'}]",
        "total_net": "1457.0",
    },
    "case 4": {
        "id_card": "2222222222222",
        "title": "นางสาว",
        "first_name": "สมศรี",
        "last_name": "มีสุข",
        "birth_date": "1973-01-01",
        "visit_date": "2024-08-20",
        "treatment_type": "อุบัติเหตุ",
        "visit_type": "เข้ารับการรักษาต่อเนื่อง",
        "chief_complaint": "\r\nนัดเอ็กซเรย์ก่อนพบแพทย์//ไม่แพ้ยา//อัมพรRN",
        "diagnosis": "S925 กระดูกนิ้วเท้าอื่นหัก (Fracture of other toe)",
        "billings": "[{'billing_initial': '185.00', 'billing_discount': '0.00', 'simb_billing_code': '1.1.14(2)', 'billing_net_amount': '185.00', 'local_billing_code': '280', 'local_billing_name': 'Other Medical Service Charge ค่าบริการทางการแพทย์'}, {'billing_initial': '265.00', 'billing_discount': '0.00', 'simb_billing_code': '1.1.2(1)', 'billing_net_amount': '265.00', 'local_billing_code': '70', 'local_billing_name': 'Medical Supplies 1   เวชภัณฑ์1'}, {'billing_initial': '1060.00', 'billing_discount': '0.00', 'simb_billing_code': '1.1.5(1)', 'billing_net_amount': '1060.00', 'local_billing_code': '130', 'local_billing_name': 'Standard Diagnostic    ค่าตรวจรังสีปกติ  รังสีเอ็กซ์'}, {'billing_initial': '720', 'billing_discount': '0', 'simb_billing_code': '1.2.1(1)', 'billing_net_amount': '720.00', 'local_billing_code': '310', 'local_billing_name': 'First Outpatient care before admission  ค่าแพทย์ตรวจรักษาครั้งแรก'}, {'billing_initial': '318.00', 'billing_discount': '0.00', 'simb_billing_code': '1.2.1(11)', 'billing_net_amount': '318.00', 'local_billing_code': '410', 'local_billing_name': 'Special Diagnositics Physician Service  ค่าตรวจวินิจฉัยสาขาเฉพาะ ค่าวิเคราะห์ภาพการแพทย์ คลื่นหัวใจ และอื่นๆ '}, {'billing_initial': '600', 'billing_discount': '0', 'simb_billing_code': '1.2.2', 'billing_net_amount': '600.00', 'local_billing_code': '430', 'local_billing_name': 'Physician Procedures  ค่าแพทย์ศัลยกรรม  หัตถการผ่าตัด'}, {'billing_initial': '435', 'billing_discount': '43', 'simb_billing_code': '1.1.1(3)', 'billing_net_amount': '392.00', 'local_billing_code': '50', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'billing_initial': '185', 'billing_discount': '0', 'simb_billing_code': '1.1.12', 'billing_net_amount': '185.00', 'local_billing_code': '240', 'local_billing_name': 'Nursing and Midwifery Charge   ค่าบริการพยาบาล'}]",
        "order_items": "[{'initial': '185', 'item_id': 'NH', 'discount': '0', 'item_name': 'ค่าบริการโรงพยาบาล OPD (OPD Hospital Service Charge)', 'net_amount': '185.00', 'item_amount': '1', 'simb_billing_code': '1.1.14(2)', 'local_billing_code': '280', 'local_billing_name': 'Other Medical Service Charge ค่าบริการทางการแพทย์'}, {'initial': '0', 'item_id': 'APPT', 'discount': '0', 'item_name': 'นัดหมาย (Appointment)', 'net_amount': '0.00', 'item_amount': '1', 'simb_billing_code': '1.1.14(2)', 'local_billing_code': '280', 'local_billing_name': 'Other Medical Service Charge ค่าบริการทางการแพทย์'}, {'initial': '139', 'item_id': '2029COBA0000001', 'discount': '0', 'item_name': 'ELASTIC BANDAGE 1\" (COBAN 1\") #', 'net_amount': '139.00', 'item_amount': '1', 'simb_billing_code': '1.1.2(1)', 'local_billing_code': '70', 'local_billing_name': 'Medical Supplies 1   เวชภัณฑ์1'}, {'initial': '68', 'item_id': '2030GAUZ2208005', 'discount': '0', 'item_name': 'GAUZE PAD STERILE 2  X 2  (5 ชิ้น/ห่อ)', 'net_amount': '68.00', 'item_amount': '1', 'simb_billing_code': '1.1.2(1)', 'local_billing_code': '70', 'local_billing_name': 'Medical Supplies 1   เวชภัณฑ์1'}, {'initial': '58', 'item_id': '2041GLOVSTNP00M', 'discount': '0', 'item_name': 'GLOVE # M ไม่มีแป้ง.(ศรีตรังกล่องเขียว)', 'net_amount': '58.00', 'item_amount': '1', 'simb_billing_code': '1.1.2(1)', 'local_billing_code': '70', 'local_billing_name': 'Medical Supplies 1   เวชภัณฑ์1'}, {'initial': '530', 'item_id': 'XL0060', 'discount': '0', 'item_name': 'LT.FOURTH TOE AP', 'net_amount': '530.00', 'item_amount': '1', 'simb_billing_code': '1.1.5(1)', 'local_billing_code': '130', 'local_billing_name': 'Standard Diagnostic    ค่าตรวจรังสีปกติ  รังสีเอ็กซ์'}, {'initial': '530', 'item_id': 'XL0061', 'discount': '0', 'item_name': 'LT.FOURTH TOE OBLIQUE', 'net_amount': '530.00', 'item_amount': '1', 'simb_billing_code': '1.1.5(1)', 'local_billing_code': '130', 'local_billing_name': 'Standard Diagnostic    ค่าตรวจรังสีปกติ  รังสีเอ็กซ์'}, {'initial': '720', 'item_id': 'DF1', 'discount': '0', 'item_name': 'DF แพทย์ตรวจรักษา (Doctor Fee)', 'net_amount': '720.00', 'item_amount': '1', 'simb_billing_code': '1.2.1(1)', 'local_billing_code': '310', 'local_billing_name': 'First Outpatient care before admission  ค่าแพทย์ตรวจรักษาครั้งแรก'}, {'initial': '159', 'item_id': 'XL0060', 'discount': '0', 'item_name': 'LT.FOURTH TOE AP', 'net_amount': '159.00', 'item_amount': '1', 'simb_billing_code': '1.2.1(11)', 'local_billing_code': '410', 'local_billing_name': 'Special Diagnositics Physician Service  ค่าตรวจวินิจฉัยสาขาเฉพาะ ค่าวิเคราะห์ภาพการแพทย์ คลื่นหัวใจ และอื่นๆ '}, {'initial': '159', 'item_id': 'XL0061', 'discount': '0', 'item_name': 'LT.FOURTH TOE OBLIQUE', 'net_amount': '159.00', 'item_amount': '1', 'simb_billing_code': '1.2.1(11)', 'local_billing_code': '410', 'local_billing_name': 'Special Diagnositics Physician Service  ค่าตรวจวินิจฉัยสาขาเฉพาะ ค่าวิเคราะห์ภาพการแพทย์ คลื่นหัวใจ และอื่นๆ '}, {'initial': '600', 'item_id': 'DFO-O-BUDDY', 'discount': '0', 'item_name': 'BUDDY SPINT.', 'net_amount': '600.00', 'item_amount': '1', 'simb_billing_code': '1.2.2', 'local_billing_code': '430', 'local_billing_name': 'Physician Procedures  ค่าแพทย์ศัลยกรรม  หัตถการผ่าตัด'}, {'initial': '435', 'item_id': '0111D07C1250L07', 'discount': '43', 'item_name': '==  1250 MG.==  CALCIUM CARBONATE', 'net_amount': '392.00', 'item_amount': '30', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': '50', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '185', 'item_id': 'NSE', 'discount': '0', 'item_name': 'ค่าบริการพยาบาล ผู้ป่วยนอก (Out Patient Service Charge)', 'net_amount': '185.00', 'item_amount': '1', 'simb_billing_code': '1.1.12', 'local_billing_code': '240', 'local_billing_name': 'Nursing and Midwifery Charge   ค่าบริการพยาบาล'}]",
        "total_net": "3725.0",
    },
    "case 5": {
        "id_card": "3333333333333",
        "title": "นาย",
        "first_name": "สมบุก",
        "last_name": "สมบัน",
        "birth_date": "1973-01-01",
        "visit_date": "2024-08-20",
        "treatment_type": "เจ็บป่วย",
        "visit_type": "เข้ารับการรักษาต่อเนื่อง",
        "chief_complaint": "ปวดศีรษะ",
        "diagnosis": "K30 อาหารไม่ย่อย (Functional Dyspepsia)",
        "billings": "[{'billing_initial': '1455.00', 'billing_discount': '146.00', 'simb_billing_code': '1.1.1(3)', 'billing_net_amount': '1309.00', 'local_billing_code': '11010003', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'billing_initial': '15.00', 'billing_discount': '0.00', 'simb_billing_code': '1.1.2(1)', 'billing_net_amount': '15.00', 'local_billing_code': '11020001', 'local_billing_name': 'Medical Supplies 1   เวชภัณฑ์1'}, {'billing_initial': '300', 'billing_discount': '0', 'simb_billing_code': '1.2.1(1)', 'billing_net_amount': '300.00', 'local_billing_code': '12010001', 'local_billing_name': 'First Outpatient care before admission  ค่าแพทย์ตรวจรักษาครั้งแรก'}]",
        "order_items": "[{'initial': '120', 'item_id': 'P05C07XT04', 'discount': '12', 'item_name': 'ORPHENADRINE/PARACETAMOL (O, NORGESIC)', 'net_amount': '108.00', 'item_amount': '15', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': '11010003', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '525', 'item_id': 'P04L07DT08', 'discount': '53', 'item_name': 'IBUPROFEN ARGININE (400 MG) (O, SPEDIFEN)', 'net_amount': '472.00', 'item_amount': '15', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': '11010003', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '120', 'item_id': 'P04K11NE01', 'discount': '12', 'item_name': 'AROTIKA RUB (35 GM)', 'net_amount': '108.00', 'item_amount': '1', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': '11010003', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '690', 'item_id': 'P04L07CI02', 'discount': '69', 'item_name': 'PARECOXIB (40 MG/2 ML) INJ (O, DYNASTAT)', 'net_amount': '621.00', 'item_amount': '1', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': '11010003', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '5', 'item_id': '106004    ', 'discount': '0', 'item_name': 'SYRINGE 3  ML.', 'net_amount': '5.00', 'item_amount': '1', 'simb_billing_code': '1.1.2(1)', 'local_billing_code': '11020001', 'local_billing_name': 'Medical Supplies 1   เวชภัณฑ์1'}, {'initial': '5', 'item_id': '105018    ', 'discount': '0', 'item_name': 'NEEDLE DISP. #18*1 1/2 inch', 'net_amount': '5.00', 'item_amount': '1', 'simb_billing_code': '1.1.2(1)', 'local_billing_code': '11020001', 'local_billing_name': 'Medical Supplies 1   เวชภัณฑ์1'}, {'initial': '5', 'item_id': '105023    ', 'discount': '0', 'item_name': 'NEEDLE DISP. #24*1 1/2 inch', 'net_amount': '5.00', 'item_amount': '1', 'simb_billing_code': '1.1.2(1)', 'local_billing_code': '11020001', 'local_billing_name': 'Medical Supplies 1   เวชภัณฑ์1'}, {'initial': '300', 'item_id': 'DRFEE     ', 'discount': '0', 'item_name': '[DF] ตรวจรักษากรณีผู้ป่วยนอก ครั้งแรก', 'net_amount': '300.00', 'item_amount': '300', 'simb_billing_code': '1.2.1(1)', 'local_billing_code': '12010001', 'local_billing_name': 'First Outpatient care before admission  ค่าแพทย์ตรวจรักษาครั้งแรก'}]",
        "total_net": "1624.0",
    },
    "case 6": {
        "id_card": "4444444444444",
        "title": "นาย",
        "first_name": "สมบุก",
        "last_name": "สมบัน",
        "birth_date": "1973-01-01",
        "visit_date": "2024-08-20",
        "treatment_type": "เจ็บป่วย",
        "visit_type": "เข้ารับการรักษาต่อเนื่อง",
        "chief_complaint": "พบก้อนผิดปกติบริเวณปอด",
        "diagnosis": "C34.1 มะเร็งปอด (Malignant Neoplasm of Upper Lobe, Bronchus or Lung)",
        "billings": "[{'billing_initial': '15000.00', 'billing_discount': '1500.00', 'simb_billing_code': '2.1.1(3)', 'billing_net_amount': '13500.00', 'local_billing_code': '21010001', 'local_billing_name': 'Chemotherapy   ค่ารักษาเคมีบำบัด'}, {'billing_initial': '5000.00', 'billing_discount': '500.00', 'simb_billing_code': '2.1.2(1)', 'billing_net_amount': '4500.00', 'local_billing_code': '21020001', 'local_billing_name': 'Radiation Therapy   ค่ารักษารังสีบำบัด'}]",
        "order_items": "[{'initial': '3000.00', 'item_id': 'P06C08XT01', 'discount': '300.00', 'item_name': 'Cisplatin (Injection)', 'net_amount': '2700.00', 'item_amount': '1', 'simb_billing_code': '2.1.1(3)', 'local_billing_code': '21010001', 'local_billing_name': 'Chemotherapy   ค่ารักษาเคมีบำบัด'}, {'initial': '1500.00', 'item_id': 'P06C08XT02', 'discount': '150.00', 'item_name': 'Etoposide (Injection)', 'net_amount': '1350.00', 'item_amount': '1', 'simb_billing_code': '2.1.1(3)', 'local_billing_code': '21010001', 'local_billing_name': 'Chemotherapy   ค่ารักษาเคมีบำบัด'}]",
        "total_net": "18000"
    }
}

policies = {
    "Approve ✅": {
        "effective_date": "2024-01-01",
        "expire_date": "2024-12-31",
        "insurer_name": "บริษัทประกัน A",
        "insurer_contact": "012-345-6789",
        "policy_name": "ประกันสุขภาพรายบุคคล",
        "policy_number": "1234567890",
        "holder_cid": "0123456789012",
        "holder_name": "สมชาย ใจดี",
        "service_type": "OPD",
        "claim_date": "2024-08-05",
        "opd_remaining": "10000",
        "general_exclusions": "โรคที่เป็นมาก่อนการทำประกันในปีแรก, การทำศัลยกรรมเพื่อความสวยงาม",
        "specific_exclusions": "การรักษาทางทันตกรรมที่ไม่ได้เกิดจากอุบัติเหตุ, ยาที่ไม่มีใบสั่งแพทย์"
    },
    "uncoverage period 🗓️": {
        "effective_date": "2025-01-01",
        "expire_date": "2025-12-31",
        "insurer_name": "บริษัทประกัน B",
        "insurer_contact": "098-765-4321",
        "policy_name": "ประกันอุบัติเหตุรายบุคคล",
        "policy_number": "0987654321",
        "holder_cid": "9876543210987",
        "holder_name": "สวยใส ดีใจ",
        "service_type": "OPD",
        "claim_date": "2024-09-05",
        "opd_remaining": "10000",
        "general_exclusions": "การบาดเจ็บจากการเล่นกีฬา",
        "specific_exclusions": "การบาดเจ็บจากการปีนเขา"
    },
    "opd exceeds 🪙": {
        "effective_date": "2025-01-01",
        "expire_date": "2025-12-31",
        "insurer_name": "บริษัทประกัน B",
        "insurer_contact": "098-765-4321",
        "policy_name": "ประกันอุบัติเหตุรายบุคคล",
        "policy_number": "0987654321",
        "holder_cid": "9876543210987",
        "holder_name": "กุ๊กกุ๊ก ไก๊ไก่",
        "service_type": "OPD",
        "claim_date": "2025-09-05",
        "opd_remaining": "1000",
        "general_exclusions": "การบาดเจ็บจากการเล่นกีฬา",
        "specific_exclusions": "การบาดเจ็บจากการปีนเขา"
    },
    "uncoverage treatment type 🚑": {
        "effective_date": "2025-01-01",
        "expire_date": "2025-12-31",
        "insurer_name": "บริษัทประกัน B",
        "insurer_contact": "098-765-4321",
        "policy_name": "ประกันสุขภาพรายบุคคล",
        "policy_number": "0987654321",
        "holder_cid": "9876543210987",
        "holder_name": "สมศรี มีสุข",
        "service_type": "OPD",
        "claim_date": "2025-09-05",
        "opd_remaining": "10000",
        "general_exclusions": "การบาดเจ็บจากการเล่นกีฬา",
        "specific_exclusions": "การบาดเจ็บจากการปีนเขา"
    },
    "wait symptoms🩺": {
        "effective_date": "2025-01-01",
        "expire_date": "2025-12-31",
        "insurer_name": "บริษัทประกัน B",
        "insurer_contact": "098-765-4321",
        "policy_name": "ประกันสุขภาพรายบุคคล",
        "policy_number": "0987654321",
        "holder_cid": "9876543210987",
        "holder_name": "สมศรี มีสุข",
        "service_type": "OPD",
        "claim_date": "2025-09-05",
        "opd_remaining": "10000",
        "general_exclusions": "การบาดเจ็บจากการเล่นกีฬา",
        "specific_exclusions": "การบาดเจ็บจากการปีนเขา"
    }
    ,
    "chronic disease🫁": {
        "effective_date": "2025-01-01",
        "expire_date": "2025-12-31",
        "insurer_name": "บริษัทประกัน B",
        "insurer_contact": "098-765-4321",
        "policy_name": "ประกันสุขภาพรายบุคคล",
        "policy_number": "0987654321",
        "holder_cid": "9876543210987",
        "holder_name": "สมบุก สมบัน",
        "service_type": "OPD",
        "claim_date": "2025-09-05",
        "opd_remaining": "100000",
        "general_exclusions": "โรคร้ายแรง",
        "specific_exclusions": "มะเร็ง เนื้องอก"
    }
}

## populate info

In [86]:
def populate_data(selected_patient, selected_policy):
    patient_data = patients.get(selected_patient, {})
    policy_data = policies.get(selected_policy, {})
    return (
        # Patient data
        patient_data.get("id_card", ""),
        f"{patient_data.get('title', '')} {patient_data.get('first_name', '')} {patient_data.get('last_name', '')}",
        patient_data.get("birth_date", ""),
        patient_data.get("visit_date", ""),
        patient_data.get("visit_type", ""),
        patient_data.get("treatment_type", ""),
        patient_data.get("diagnosis", ""),
        patient_data.get("chief_complaint", ""),
        patient_data.get("billings", ""),
        patient_data.get("order_items", ""),
        patient_data.get("total_net", ""),
        # Policy data
        policy_data.get("effective_date", ""),
        policy_data.get("expire_date", ""),
        policy_data.get("insurer_name", ""),
        policy_data.get("insurer_contact", ""),
        policy_data.get("policy_name", ""),
        policy_data.get("policy_number", ""),
        policy_data.get("holder_cid", ""),
        policy_data.get("holder_name", ""),
        policy_data.get("service_type", ""),
        policy_data.get("claim_date", ""),
        policy_data.get("opd_remaining", ""),
        policy_data.get("general_exclusions", ""),
        policy_data.get("specific_exclusions", "")
    )

combined_options = [
    {"label": f"{patient} - {policy}", "patient": patient, "policy": policy}
    for patient, policy in zip(patients.keys(), policies.keys())
]

def populate_data_combined(selected_option):
    # Find the selected patient and policy
    selected_data = next((item for item in combined_options if item["label"] == selected_option), {})
    selected_patient = selected_data.get("patient", "")
    selected_policy = selected_data.get("policy", "")
    return populate_data(selected_patient, selected_policy)

### condition

In [108]:
def uncoverage_chronic(diagnosis, general_exclusions):
    prompt_uncoverage_chronic = f"""
คุณเป็นนักวิเคราะห์ประกันภัยที่มีหน้าที่ตรวจสอบว่าการวินิจฉัยทางการแพทย์อยู่ในเงื่อนไขการยกเว้นทั่วไปจากกรมธรรม์หรือไม่ เงื่อนไขการยกเว้นทั่วไปมีดังนี้: {general_exclusions}

กรุณาวิเคราะห์การวินิจฉัยที่ให้มาและตรวจสอบว่าตรงกับหมวดหมู่ใดในเงื่อนไขการยกเว้นข้างต้นหรือไม่ หากตรงกับเงื่อนไขใด ให้ระบุว่า "เข้ากลุ่มประกัน" หากไม่ตรงกับเงื่อนไขใดเลย ให้ระบุว่า "ไม่เข้ากลุ่มใด"

**ลำดับความคิด (Chain of Thoughts):**
1. ระบุคำสำคัญหรือวลีสำคัญในข้อความการวินิจฉัยที่เกี่ยวข้องกับเงื่อนไขการยกเว้น
2. เปรียบเทียบคำสำคัญที่ระบุกับเงื่อนไขการยกเว้นทั่วไปที่ให้มา
3. ตรวจสอบว่าการวินิจฉัยตรงกับหมวดหมู่ใดในเงื่อนไขการยกเว้น
4. สรุปผลลัพธ์เป็น "เข้ากลุ่มประกัน" หรือ "ไม่เข้ากลุ่มใด"
5. ห้ามพูดถึงสิ่งอื่นๆที่ไม่เกี่ยว

**ตัวอย่างการวิเคราะห์:**

**ตัวอย่างที่ 1:**
การวินิจฉัย: "ผู้ป่วยได้รับการวินิจฉัยว่าเป็นมะเร็งปอด"
เงื่อนไขการยกเว้น: ["โรคร้ายแรง", "การศัลยกรรมหรือการรักษาความงาม", "การบาดเจ็บจากกิจกรรมผิดกฎหมาย"]
คำตอบที่ถูกต้อง:
- ผลลัพธ์: "เข้ากลุ่มประกัน"
- การวิเคราะห์: การวินิจฉัยระบุว่า "มะเร็งปอด" ซึ่งเป็นโรคที่คุกคามชีวิตและเข้าข่ายในหมวดหมู่ "โรคร้ายแรง"


**ตัวอย่างที่ 2:**
การวินิจฉัย: "ผู้ป่วยเข้ารับการเสริมจมูก"
เงื่อนไขการยกเว้น: ["โรคร้ายแรง", "การศัลยกรรมหรือการรักษาความงาม", "การบาดเจ็บจากกิจกรรมผิดกฎหมาย"]
คำตอบที่ถูกต้อง:
- ผลลัพธ์: "เข้ากลุ่มประกัน"
- การวิเคราะห์: การวินิจฉัยระบุว่า "เสริมจมูก" ซึ่งเข้าข่ายหมวดหมู่ "การศัลยกรรมหรือการรักษาความงาม"


**ตัวอย่างที่ 3:**
การวินิจฉัย: "ไข้หวัดทั่วไป"
เงื่อนไขการยกเว้น: ["โรคร้ายแรง", "การศัลยกรรมหรือการรักษาความงาม", "การบาดเจ็บจากกิจกรรมผิดกฎหมาย"]
คำตอบที่ถูกต้อง:
- ผลลัพธ์: "ไม่เข้ากลุ่มใด"
- การวิเคราะห์: การวินิจฉัยระบุว่า "ไข้หวัดทั่วไป" ซึ่งเป็นอาการเจ็บป่วยปกติและไม่คุกคามชีวิต

**งานของคุณ:**

การวินิจฉัย: {diagnosis}
เงื่อนไขการยกเว้น: {general_exclusions}
คำตอบที่ถูกต้อง:
"""
    reasons = ""
    formatted_prompt = prompt_uncoverage_chronic.format(diagnosis=diagnosis, general_exclusions=general_exclusions)
    pattern = r"\b(ไม่เข้ากลุ่มใด|เข้ากลุ่มประกัน)\b"
    cond_output = generate_response(formatted_prompt, model_param=model_param03, url=url)
    match = re.search(pattern, cond_output)

    if match:
        con_re = match.group()
        if con_re  == "ไม่เข้ากลุ่มใด":
            return True, reasons
        elif con_re == "เข้ากลุ่มประกัน":
            reasons  = f"Policy exclusion conditions show uncoverage {general_exclusions}"
            return False, reasons
    else:
        reasons = "cannot Processs"
        return False, reasons

def check_180diseased(icd10_code, diagnosis):
    diagnosis = str(diagnosis)
    icd10_code = diagnosis.split(' ')[0] if diagnosis else None
    symptom = {"K30", "K297", "I495"}
    if icd10_code in symptom:
        result = False
        reason = f"{diagnosis} is a symptom or disease with a waiting period of 180 days"
        return result, reason
    return True, ""

def uncoverage_period(claim_date, effective_date, expire_date):
    try:
        claim_date_obj = datetime.strptime(claim_date, "%Y-%m-%d")
        effective_date_obj = datetime.strptime(effective_date, "%Y-%m-%d")
        expire_date_obj = datetime.strptime(expire_date, "%Y-%m-%d")

        if claim_date_obj < effective_date_obj or claim_date_obj > expire_date_obj:
            return False, f"Claim date {claim_date} is outside the policy coverage period ({effective_date} to {expire_date})."
        return True, ""
    except ValueError:
        return False, "Invalid date format."

def opd_exceeds(opd_remaining, total_net):
    try:
        opd_remaining = float(opd_remaining)
        total_net = float(total_net)
    except ValueError:
        return False, "Invalid number format."
    if total_net > opd_remaining:
        return False, f"Total Billing {total_net} exceeds remaining OPD limit {opd_remaining}."
    return True, ""

def check_and_classify_treatment_type(treatment_type, policy_name):
    # สร้าง prompt สำหรับการจัดกลุ่มประเภทการรักษา
    treatment_type_prompt = f"""
    จัดกลุ่มประเภทการเข้ารับการรักษาตามตัวเลือกต่อไปนี้:
    1. เจ็บป่วย: ปัญหาด้านสุขภาพหรือโรคที่ต้องการการรักษา เช่น ไข้หวัด ไข้หวัดใหญ่ หรืออาการเจ็บป่วยทั่วไป
    2. อุบัติเหตุ: การบาดเจ็บหรือเหตุการณ์ฉุกเฉินที่เกิดจากอุบัติเหตุ เช่น อุบัติเหตุทางรถยนต์ การลื่นล้ม หรือการบาดเจ็บทางกาย
    ประเภทการรักษา: {treatment_type}
    ผลลัพธ์: [สุขภาพ/อุบัติเหตุ]
    หากไม่สามารถจัดกลุ่มได้ ให้ตอบกลับด้วย "ไม่ทราบ"
    หลังจากตอบเสร็จให้ลงท้ายด้วยว่า /end
    """

    vt_output = generate_response(treatment_type_prompt, model_param=model_param02, url=url2)
    if "ไม่ทราบ" in vt_output:
        return False, f"Unable to group treatment type: {treatment_type}. Please check the information."
    pattern = r"\b(สุขภาพ|อุบัติเหตุ)\b"
    match = re.search(pattern, vt_output)
    classified_treatment_type = match.group() if match else "ไม่ทราบ"

    try:
        normalized_treatment_type = classified_treatment_type.strip().lower()
        normalized_policy_name = policy_name.strip().lower()
        health_keywords = ["สุขภาพ", "รายบุคคล", "กลุ่ม"]
        accident_keywords = ["อุบัติเหตุ"]
        if normalized_treatment_type == "สุขภาพ":
            if any(keyword in normalized_policy_name for keyword in health_keywords):
                return True, ""
            else:
                return False, f"Accident insurance cannot be used for medical treatment."
        elif normalized_treatment_type == "อุบัติเหตุ":
            if any(keyword in normalized_policy_name for keyword in accident_keywords):
                return True, ""
            else:
                return False, f"Health insurance cannot be used for accident treatment."
        else:
            return False, f"Treatment type {treatment_type} and insurance policy type {policy_name} do not match."

    except AttributeError:
        return False, "Invalid input types provided"


## claim validation

In [105]:
# รวมการเช็คเงื่อนไขทั้งหมด
def validate_claim(inputs):
    claim_date, effective_date, expire_date = inputs["claim_date"], inputs["effective_date"], inputs["expire_date"]
    opd_remaining, total_net = inputs["opd_remaining"], inputs["total_net"]
    treatment_type, policy_name = inputs["treatment_type"], inputs["policy_name"]
    diagnosis = inputs["diagnosis"]
    general_exclusions = inputs["general_exclusions"]

    conditions = {
        "period_check": uncoverage_period(claim_date, effective_date, expire_date),
        "opd_check": opd_exceeds(opd_remaining, total_net),
        "treatment_type_check": check_and_classify_treatment_type(treatment_type, policy_name),
        "symptom_check": check_180diseased(None, diagnosis),
        "chronic_check": uncoverage_chronic(diagnosis, general_exclusions)
    }

    status = "Approve ✅"
    reasons = []

    # Update checkboxes and reasons
    for key, (condition_met, reason) in conditions.items():
        if not condition_met:
            status = "Decline ❌"
            reasons.append(reason)
            conditions[key] = True  # Tick checkbox
        else:
            conditions[key] = False  # Untick checkbox

    return status, "\n".join(reasons), conditions

In [106]:
# ฟังก์ชัน Gradio สำหรับการกดปุ่ม
def on_submit(
    claim_date, effective_date, expire_date, opd_remaining, total_net, treatment_type, policy_name, diagnosis, general_exclusions
):
    inputs = {
        "claim_date": claim_date,
        "effective_date": effective_date,
        "expire_date": expire_date,
        "opd_remaining": opd_remaining,
        "total_net": total_net,
        "treatment_type": treatment_type,
        "policy_name": policy_name,
        "diagnosis": diagnosis,
        "general_exclusions": general_exclusions
    }

    status, reasons, conditions = validate_claim(inputs)

    return (
        status,
        reasons,
        conditions["period_check"],
        conditions["opd_check"],
        conditions["treatment_type_check"],
        conditions["symptom_check"],
        conditions["chronic_check"]
    )

## gradio

In [107]:
# Create Gradio interface
with gr.Blocks() as demo:
    with gr.Row():
        select = gr.Dropdown(
            choices=[option["label"] for option in combined_options],
            label="เลือกผู้ป่วยและกรมธรรม์",
            value="",
            interactive=True
        )
    completion_status = gr.Textbox(label="สถานะการทำงาน", interactive=False, visible=False)
    with gr.Row():
        with gr.Column():
            gr.Markdown("### ข้อมูลผู้ป่วย")
            id_card_output = gr.Textbox(label="เลขบัตรประชาชน", interactive = True)
            with gr.Row():
                name_output = gr.Textbox(label="ชื่อ-นามสกุล", interactive = True)
                birth_date_output = gr.Textbox(label="วันเดือนปีเกิด", placeholder="yyy-mm-dd", interactive = True)
                visit_date_output = gr.Textbox(label="วันที่เข้ารับการรักษา", placeholder="yyy-mm-dd", interactive = True)
            gr.Markdown("### ข้อมูลการวินิจฉัย")
            with gr.Row():
                treatment_type_output = gr.Dropdown(label="ประเภทการเข้ารักษา", choices=["เจ็บป่วย", "อุบัติเหตุ", "ทันตกรรม"], interactive = True)
                visit_type_output = gr.Dropdown(label="ประเภทการรักษา", choices=["เข้ารับการรักษาครั้งแรก", "เข้ารับการรักษาต่อเนื่อง"], interactive = True)
            with gr.Row():
                chief_complaint_output = gr.Textbox(label="อาการสำคัญ", interactive = True)
                diagnosis_output = gr.Textbox(label="การวินิจฉัย", interactive = True)
            gr.Markdown("### ข้อมูลค่ารักษาพยาบาล")
            with gr.Row():
                billings_output = gr.Textbox(label="รายการค่ารักษาพยาบาล", max_lines = 3, interactive = True)
                order_items_output = gr.Textbox(label="รายการยา/อุปกรณ์การรักษา", max_lines = 3, interactive = True)
            total_net_output = gr.Textbox(label="รวมค่าใช้จ่ายสุทธิ", interactive = True)
        with gr.Column():
            gr.Markdown("### ข้อมูลผู้ถือกรมธรรม์")
            with gr.Row():
                holder_cid_output = gr.Textbox(label="เลขบัตรประชาชนของผู้ถือกรมธรรม์", interactive = True)
                holder_name_output = gr.Textbox(label="ชื่อผู้ถือกรมธรรม์", interactive = True)
                claim_date_output = gr.Textbox(label="วันที่เคลม", interactive = True)
            gr.Markdown("### ข้อมูลประกัน")
            with gr.Row():
                insurer_name_output = gr.Textbox(label="ชื่อบริษัทประกัน", interactive = True)
                insurer_contact_output = gr.Textbox(label="ติดต่อผู้ให้ประกัน", interactive = True)
                policy_number_output = gr.Textbox(label="เลขที่กรมธรรม์", interactive = True)
            with gr.Row():
                policy_name_output = gr.Dropdown(label="แผนประกัน", choices=["ประกันสุขภาพรายบุคคล", "ประกันอุบัติเหตุรายบุคคล", "ประกันสุขภาพกลุ่ม", "ประกันอุบัติเหตุกลุ่ม"], interactive = True)
                service_type_output = gr.Dropdown(label="ประเภทบริการ", choices=["OPD", "IPD"], interactive = True)
            gr.Markdown("### ข้อมูลผลประโยชน์ความคุ้มครอง")
            with gr.Row():
                effective_date_output = gr.Textbox(label="วันที่กรมธรรม์มีผลบังคับ", interactive = True)
                expire_date_output = gr.Textbox(label="วันที่กรมธรรม์สิ้นสุดความคุ้มครอง", interactive = True)
            with gr.Row():
                opd_remaining_output = gr.Textbox(label="วงเงิน OPD คงเหลือ", interactive = True)
                general_exclusions_output = gr.Textbox(label="ข้อยกเว้นทั่วไป", interactive = True)
                specific_exclusions_output = gr.Textbox(label="ข้อยกเว้นเฉพาะ", interactive = True)

    # Bind combined selection to populate data
    select.change(
        populate_data_combined,
        inputs=[select],
        outputs=[
            id_card_output, name_output, birth_date_output, visit_date_output,
            visit_type_output, treatment_type_output, diagnosis_output, chief_complaint_output,
            billings_output, order_items_output, total_net_output, effective_date_output,
            expire_date_output, insurer_name_output, insurer_contact_output, policy_name_output,
            policy_number_output, holder_cid_output, holder_name_output, service_type_output, claim_date_output,
            opd_remaining_output, general_exclusions_output, specific_exclusions_output
        ]
    )
    submit_button = gr.Button("Claim Validate", interactive=True)
    gr.Markdown("### เช็คเงื่อนไขการเคลมแต่ละเงื่อนไข")
    with gr.Row():
        period_check = gr.Checkbox(label="Uncoverage Period", visible=True, interactive=False)
        treatment_type_check = gr.Checkbox(label="Uncoverage Type", visible=True, interactive=False)
        symptom_check = gr.Checkbox(label="Uncoverage Symptom", visible=True, interactive=False)
        chronic_check = gr.Checkbox(label="Uncoverage Chronic", visible=True, interactive=False)
        duplicate_check = gr.Checkbox(label="Duplicate Claim", visible=True, interactive=False)
        opd_check = gr.Checkbox(label="OPD Exceeds", visible=True, interactive=False)
        drug_check = gr.Checkbox(label="Check Drug", visible=True, interactive=False)
        overprice_check = gr.Checkbox(label="Item Overprice", visible=True, interactive=False)
    with gr.Row():
        coverage_status_output = gr.Textbox(label="สถานะความคุ้มครอง", interactive=False)
        uncoverage_reasons_output = gr.Textbox(label="เหตุผลที่ไม่สามารถเคลมได้", interactive=False)

    submit_button.click(
    on_submit,
    inputs=[
        claim_date_output, effective_date_output, expire_date_output,
        opd_remaining_output, total_net_output, treatment_type_output, policy_name_output,
        diagnosis_output,  general_exclusions_output
    ],
    outputs=[
        coverage_status_output, uncoverage_reasons_output,
        period_check, opd_check, treatment_type_check,
        symptom_check, chronic_check
    ]
)
demo.launch(debug=True)

/usr/local/lib/python3.10/dist-packages/gradio/components/dropdown.py:226: UserWarning: The value passed into gr.Dropdown() is not in the list of choices. Please update the list of choices to include:  or set allow_custom_value=True.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://a68911d4fc4d3cffce.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://a68911d4fc4d3cffce.gradio.live
